# Getting Player Info

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

KeyboardInterrupt: 

In [1]:
import requests

alphabet = [chr(i) for i in range(ord('a'), ord('z') + 1)]

url_player_info = "https://www.basketball-reference.com/players/{}/"
for letter in alphabet:
    url = url_player_info.format(letter)
    data = requests.get(url)
    with open ('player_info/{}.html'.format(letter), 'w') as f:
        f.write(data.text)

In [61]:
from bs4 import BeautifulSoup
import pandas as pd

dfs = []

for letter in alphabet:
    with open('player_info/{}.html'.format(letter)) as f:
        page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    players_html = soup.find(id="players")
    players_table = pd.read_html(str(players_html))[0]
    # Drop rows that have Player as 'Player' after the header row
    players_table = players_table[players_table['Player'] != 'Player']
    dfs.append(players_table)
    
players = pd.concat(dfs)
players.head()

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"


In [62]:
# Remove asterisk from player names
players['Player'] = players['Player'].str.replace('*', '')
# Format Birth Date to format YYYY-MM-DD
players['Birth Date'] = pd.to_datetime(players['Birth Date'])
# Change height to inches
def height_to_inches(height):
    feet, inches = height.split('-')
    return int(feet) * 12 + int(inches)
players['Ht'] = players['Ht'].apply(height_to_inches)
players.to_csv('players.csv', index=False)
players.shape
players.head()

,Player,From,To,Pos,Ht,Wt,Birth Date,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,82,240.0,1968-06-24,Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,81,235.0,1946-04-07,Iowa State
2,Kareem Abdul-Jabbar,1970,1989,C,86,225.0,1947-04-16,UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,73,162.0,1969-03-09,LSU
4,Tariq Abdul-Wahad,1998,2003,F,78,223.0,1974-11-03,"Michigan, San Jose State"


# Getting Team Info

In [11]:
url_team_info = "https://www.basketball-reference.com/teams/"
data = requests.get(url_team_info)
with open ('team_info/team_info.html', 'w') as f:
    f.write(data.text)

In [21]:
with open('team_info/team_info.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
teams_active_html = soup.find(id="all_teams_active")
#teams_defunct_html = soup.find(id="all_teams_defunct")
teams_active = pd.read_html(str(teams_active_html))[0]
#teams_active['Active'] = "Y"
#teams_defunct = pd.read_html(str(teams_defunct_html))[0]
#teams_defunct['Active'] = "N"
#teams = pd.concat([teams_active, teams_defunct])
# Drop rows that have Franchise as 'Franchise' after the header row
#teams = teams[teams['Franchise'] != 'Franchise']
#teams.head()
teams_active = teams_active[teams_active['Franchise'] != 'Franchise']
teams_active.head()
teams_active.to_csv('teams_active.csv', index=False)

In [64]:
teams.to_csv('teams.csv', index=False)

# Getting Player Statistics

### Per Game

In [2]:
years = list(range(1947, 2024))

url_player_stats_leagues = "https://www.basketball-reference.com/leagues/{}_{}_per_game.html"
url_player_stats_playoffs = "https://www.basketball-reference.com/playoffs/{}_{}_per_game.html"

In [1]:
import time
for year in years:
    print(year)
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        url = url_player_stats_leagues.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/per_game/{}_{}_per_game.html'.format(league, year), 'w') as f:
            f.write(data.text)
        url = url_player_stats_playoffs.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/per_game/{}_{}_playoffs_per_game.html'.format(league, year), 'w') as f:
            f.write(data.text)

NameError: name 'years' is not defined

In [4]:
import pandas as pd
from bs4 import BeautifulSoup
dfs_per_game = []

for year in years:
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        with open('player_stats/per_game/{}_{}_per_game.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        per_game_html = soup.find(id="per_game_stats")
        per_game_table = pd.read_html(str(per_game_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        per_game_table = per_game_table[per_game_table['Player'] != 'Player']
        per_game_table['Year'] = year
        per_game_table['Playoffs_S'] = "N"
        dfs_per_game.append(per_game_table)
        with open('player_stats/per_game/{}_{}_playoffs_per_game.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        per_game_html = soup.find(id="per_game_stats")
        per_game_table = pd.read_html(str(per_game_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        per_game_table = per_game_table[per_game_table['Player'] != 'Player']
        per_game_table['Year'] = year
        per_game_table['Playoffs_S'] = "Y"
        dfs_per_game.append(per_game_table)
        
player_stats_per_game = pd.concat(dfs_per_game)
player_stats_per_game.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Playoffs_S
0,1,John Abramovic,F,27,PIT,47,NaN,NaN,4.3,17.7,...,NaN,NaN,0.7,NaN,NaN,NaN,3.4,11.2,1947,N
1,2,Chet Aubuchon,G,30,DTF,30,NaN,NaN,0.8,3.0,...,NaN,NaN,0.7,NaN,NaN,NaN,1.5,2.2,1947,N
2,3,Norm Baker,G,23,CHS,4,NaN,NaN,0.0,0.3,...,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,1947,N
3,4,Herschel Baltimore,F,25,STB,58,NaN,NaN,0.9,4.5,...,NaN,NaN,0.3,NaN,NaN,NaN,1.7,2.4,1947,N
4,5,John Barr,F,28,STB,58,NaN,NaN,2.1,7.6,...,NaN,NaN,0.9,NaN,NaN,NaN,2.8,5.1,1947,N


In [5]:
# Remove Rk column
player_stats_per_game.drop(columns=['Rk'], inplace=True)
# Remove asterisk from player names
player_stats_per_game['Player'] = player_stats_per_game['Player'].str.replace('*', '')
player_stats_per_game.to_csv('player_stats_per_game2.csv', index=False)

### Totals

In [6]:
years = list(range(1947, 2024))

url_player_stats_leagues = "https://www.basketball-reference.com/leagues/{}_{}_totals.html"
url_player_stats_playoffs = "https://www.basketball-reference.com/playoffs/{}_{}_totals.html"

In [9]:
for year in years:
    print(year)
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        url = url_player_stats_leagues.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/totals/{}_{}_totals.html'.format(league, year), 'w') as f:
            f.write(data.text)
        url = url_player_stats_playoffs.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/totals/{}_{}_playoffs_totals.html'.format(league, year), 'w') as f:
            f.write(data.text)

1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [7]:
dfs_totals = []

for year in years:
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        with open('player_stats/totals/{}_{}_totals.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        totals_html = soup.find(id="totals_stats")
        totals_table = pd.read_html(str(totals_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        totals_table = totals_table[totals_table['Player'] != 'Player']
        totals_table['Year'] = year
        totals_table['Playoffs_S'] = "N"
        dfs_totals.append(totals_table)
        with open('player_stats/totals/{}_{}_playoffs_totals.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        totals_html = soup.find(id="totals_stats")
        totals_table = pd.read_html(str(totals_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        totals_table = totals_table[totals_table['Player'] != 'Player']
        totals_table['Year'] = year
        totals_table['Playoffs_S'] = "Y"
        dfs_totals.append(totals_table)
        
player_stats_totals = pd.concat(dfs_totals)
player_stats_totals.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Playoffs_S
0,1,John Abramovic,F,27,PIT,47,NaN,NaN,202,834,...,NaN,NaN,35,NaN,NaN,NaN,161,527,1947,N
1,2,Chet Aubuchon,G,30,DTF,30,NaN,NaN,23,91,...,NaN,NaN,20,NaN,NaN,NaN,46,65,1947,N
2,3,Norm Baker,G,23,CHS,4,NaN,NaN,0,1,...,NaN,NaN,0,NaN,NaN,NaN,0,0,1947,N
3,4,Herschel Baltimore,F,25,STB,58,NaN,NaN,53,263,...,NaN,NaN,16,NaN,NaN,NaN,98,138,1947,N
4,5,John Barr,F,28,STB,58,NaN,NaN,124,438,...,NaN,NaN,54,NaN,NaN,NaN,164,295,1947,N


In [8]:
# Remove Rk column
player_stats_totals = player_stats_totals.drop(columns=['Rk'])
# Remove asterisk from player names
player_stats_totals['Player'] = player_stats_totals['Player'].str.replace('*', '')
player_stats_totals.to_csv('player_stats_totals2.csv', index=False)

### Advanced

In [9]:
years = list(range(1947, 2024))

url_player_stats_leagues = "https://www.basketball-reference.com/leagues/{}_{}_advanced.html"
url_player_stats_playoffs = "https://www.basketball-reference.com/playoffs/{}_{}_advanced.html"

In [11]:
for year in years:
    print(year)
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        url = url_player_stats_leagues.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/advanced/{}_{}_advanced.html'.format(league, year), 'w') as f:
            f.write(data.text)
        url = url_player_stats_playoffs.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/advanced/{}_{}_playoffs_advanced.html'.format(league, year), 'w') as f:
            f.write(data.text)

1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

dfs_advanced = []

for year in years:
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        with open('player_stats/advanced/{}_{}_advanced.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        advanced_html = soup.find(id="advanced_stats")
        advanced_table = pd.read_html(str(advanced_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        advanced_table = advanced_table[advanced_table['Player'] != 'Player']
        advanced_table['Year'] = year
        advanced_table['Playoffs_S'] = "N"
        dfs_advanced.append(advanced_table)
        with open('player_stats/advanced/{}_{}_playoffs_advanced.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        advanced_html = soup.find(id="advanced_stats")
        advanced_table = pd.read_html(str(advanced_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        advanced_table = advanced_table[advanced_table['Player'] != 'Player']
        advanced_table['Year'] = year
        advanced_table['Playoffs_S'] = "Y"
        dfs_advanced.append(advanced_table)
        
player_stats_advanced = pd.concat(dfs_advanced)
player_stats_advanced.head()

,Rk,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,Year,Playoffs_S
0,1,John Abramovic,F,27,PIT,47,NaN,NaN,.289,NaN,...,-0.6,-1.7,NaN,NaN,NaN,NaN,NaN,NaN,1947,N
1,2,Chet Aubuchon,G,30,DTF,30,NaN,NaN,.305,NaN,...,1.0,1.2,NaN,NaN,NaN,NaN,NaN,NaN,1947,N
2,3,Norm Baker,G,23,CHS,4,NaN,NaN,.000,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1947,N
3,4,Herschel Baltimore,F,25,STB,58,NaN,NaN,.235,NaN,...,2.1,0.3,NaN,NaN,NaN,NaN,NaN,NaN,1947,N
4,5,John Barr,F,28,STB,58,NaN,NaN,.312,NaN,...,4.2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1947,N


In [11]:
# Remove Rk column
player_stats_advanced = player_stats_advanced.drop(columns=['Rk'])
# Remove asterisk from player names
player_stats_advanced['Player'] = player_stats_advanced['Player'].str.replace('*', '')
# Remove columsn that contain "Unnamed"
player_stats_advanced = player_stats_advanced.loc[:, ~player_stats_advanced.columns.str.contains('^Unnamed')]
player_stats_advanced.to_csv('player_stats_advanced2.csv', index=False)

### Per 100 Possesions

In [12]:
years = list(range(1974, 2024))

url_team_stats_leagues = "https://www.basketball-reference.com/leagues/{}_{}_per_poss.html"
url_team_stats_playoffs = "https://www.basketball-reference.com/playoffs/{}_{}_per_poss.html"

In [14]:
for year in years:
    print(year)
    if (year <= 1976):
        leagues = ["ABA", "NBA"]
    else:
        leagues = ["NBA"]
    for league in leagues:
        url = url_team_stats_leagues.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/per_poss/{}_{}_per_poss.html'.format(league, year), 'w') as f:
            f.write(data.text)
        url = url_team_stats_playoffs.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('player_stats/per_poss/{}_{}_playoffs_per_poss.html'.format(league, year), 'w') as f:
            f.write(data.text)

1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [13]:
dfs_per_poss = []

for year in years: 
    if (year <= 1976):
        leagues = ["ABA", "NBA"]
    else:
        leagues = ["NBA"]
    for league in leagues:
        with open('player_stats/per_poss/{}_{}_per_poss.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        per_poss_html = soup.find(id="per_poss_stats")
        per_poss_table = pd.read_html(str(per_poss_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        per_poss_table = per_poss_table[per_poss_table['Player'] != 'Player']
        per_poss_table['Year'] = year
        per_poss_table['Playoffs_S'] = "N"
        dfs_per_poss.append(per_poss_table)
        with open('player_stats/per_poss/{}_{}_playoffs_per_poss.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        per_poss_html = soup.find(id="per_poss_stats")
        per_poss_table = pd.read_html(str(per_poss_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        per_poss_table = per_poss_table[per_poss_table['Player'] != 'Player']
        per_poss_table['Year'] = year
        per_poss_table['Playoffs_S'] = "Y"
        dfs_per_poss.append(per_poss_table)
        
player_stats_per_poss = pd.concat(dfs_per_poss)
player_stats_per_poss.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,Unnamed: 29,ORtg,DRtg,Year,Playoffs_S
0,1,George Adams,SF,24,SDA,80,NaN,1433,7.8,15.5,...,1.3,0.7,3.6,3.4,17.9,NaN,102,106,1974,N
1,2,Jim Ard,C,25,MMT,27,NaN,502,6.3,15.7,...,1.5,2.4,4.0,4.2,16.7,NaN,96,103,1974,N
2,3,Bob Arnzen,SF,26,INA,20,NaN,149,7.5,15.0,...,0.9,0.6,2.5,3.4,17.5,NaN,105,105,1974,N
3,4,Bird Averitt,SG,21,SAA,74,NaN,1639,10.6,28.1,...,1.9,0.2,4.2,5.1,26.2,NaN,87,103,1974,N
4,5,Mike Barr,SG,23,VIR,45,NaN,652,5.8,12.0,...,1.9,0.3,4.2,5.6,14.0,NaN,96,107,1974,N


In [14]:
# Remove Rk column
player_stats_per_poss = player_stats_per_poss.drop(columns=['Rk'])
# Remove asterisk from player names
player_stats_per_poss['Player'] = player_stats_per_poss['Player'].str.replace('*', '')
# Remove columns that contain "Unnamed"
player_stats_per_poss = player_stats_per_poss.loc[:, ~player_stats_per_poss.columns.str.contains('^Unnamed')]
player_stats_per_poss.to_csv('player_stats_per_poss2.csv', index=False)

## Seasons

In [15]:
years = list(range(1947, 2024))

url_player_stats_leagues = "https://www.basketball-reference.com/leagues/{}_{}.html"
url_player_stats_playoffs = "https://www.basketball-reference.com/playoffs/{}_{}.html"

In [19]:
for year in years:
    print(year)
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        url = url_player_stats_leagues.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('seasons/{}_{}.html'.format(league, year), 'w') as f:
            f.write(data.text)
        url = url_player_stats_playoffs.format(league, year)
        data = requests.get(url)
        time.sleep(2.5)
        with open ('seasons/{}_{}_playoffs.html'.format(league, year), 'w') as f:
            f.write(data.text)

1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


### Team Per Game

In [129]:
dfs_teeam_stats_per_game = []

for year in years:
    print(year)
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        with open('seasons/{}_{}.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        team_stats_per_game_html = soup.find(id="per_game-team")
        team_stats_per_game_table = pd.read_html(str(team_stats_per_game_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        team_stats_per_game_table = team_stats_per_game_table[team_stats_per_game_table['Team'] != 'Team']
        team_stats_per_game_table['Year'] = year
        # For every team with an asterisk, remove the asterisk and add new column for playoff team, yes if asterisk, no if no asterisk
        team_stats_per_game_table['Playoffs'] = team_stats_per_game_table['Team'].str.contains('*', regex=False)
        team_stats_per_game_table['Team'] = team_stats_per_game_table['Team'].str.replace('*', '')
        team_stats_per_game_table['Playoffs_S'] = False
        dfs_teeam_stats_per_game.append(team_stats_per_game_table)
        with open('seasons/{}_{}_playoffs.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        team_stats_per_game_html = soup.find(id="per_game-team")
        team_stats_per_game_table = pd.read_html(str(team_stats_per_game_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        if (year >= 1998):
            team_stats_per_game_table = team_stats_per_game_table[team_stats_per_game_table['Tm'] != 'Tm']
            # Rename Tm column to Team
            team_stats_per_game_table = team_stats_per_game_table.rename(columns={'Tm': 'Team'})
        else:
            team_stats_per_game_table = team_stats_per_game_table[team_stats_per_game_table['Team'] != 'Team']
        team_stats_per_game_table['Year'] = year
        team_stats_per_game_table['Playoffs_S'] = True
        dfs_teeam_stats_per_game.append(team_stats_per_game_table)
        
team_stats_per_game = pd.concat(dfs_teeam_stats_per_game)
team_stats_per_game.head()

1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,TRB,AST,STL,BLK,TOV,PF,PTS,Year,Playoffs,Playoffs_S
0,1.0,Chicago Stags,61.0,NaN,30.8,103.4,0.298,NaN,NaN,NaN,...,NaN,7.1,NaN,NaN,NaN,24.1,77.0,1947,True,False
1,2.0,Washington Capitols,60.0,NaN,28.7,96.6,0.297,NaN,NaN,NaN,...,NaN,6.3,NaN,NaN,NaN,19.1,73.8,1947,True,False
2,3.0,Providence Steamrollers,60.0,NaN,27.2,93.0,0.292,NaN,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,20.3,72.5,1947,False,False
3,4.0,Cleveland Rebels,60.0,NaN,27.9,95.0,0.294,NaN,NaN,NaN,...,NaN,8.2,NaN,NaN,NaN,20.8,70.9,1947,True,False
4,5.0,Philadelphia Warriors,60.0,NaN,25.2,89.7,0.280,NaN,NaN,NaN,...,NaN,5.7,NaN,NaN,NaN,18.0,68.6,1947,True,False


In [130]:
# Remove Rk column
team_stats_per_game = team_stats_per_game.drop(columns=['Rk'])
team_stats_per_game.to_csv('team_stats_per_game.csv', index=False)

### Team Advanced

In [131]:
dfs_team_stats_advanced = []

for year in years:
    if (year < 1950):
        leagues = ["BAA"]
    elif (year >= 1950):
        if (year >= 1968 and year <= 1976):
            leagues = ["ABA", "NBA"]
        else:
            leagues = ["NBA"]
    for league in leagues:
        with open('seasons/{}_{}.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        soup.find('tr', class_='over_header').decompose()
        team_stats_advanced_html = soup.find(id="advanced-team")
        team_stats_advanced_table = pd.read_html(str(team_stats_advanced_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        team_stats_advanced_table = team_stats_advanced_table[team_stats_advanced_table['Team'] != 'Team']
        team_stats_advanced_table['Year'] = year
        # For every team with an asterisk, remove the asterisk and add new column for playoff team, yes if asterisk, no if no asterisk
        team_stats_advanced_table['Playoffs'] = team_stats_advanced_table['Team'].str.contains('*', regex=False)
        team_stats_advanced_table['Team'] = team_stats_advanced_table['Team'].str.replace('*', '')
        team_stats_advanced_table['Playoffs_S'] = False
        dfs_team_stats_advanced.append(team_stats_advanced_table)
        with open('seasons/{}_{}_playoffs.html'.format(league, year)) as f:
            page = f.read()
        soup = BeautifulSoup(page, 'html.parser')
        soup.find('tr', class_='over_header').decompose()
        team_stats_advanced_html = soup.find(id="advanced-team")
        team_stats_advanced_table = pd.read_html(str(team_stats_advanced_html))[0]
        # Drop rows that have Player as 'Player' after the header row
        if (year >= 1998):
            team_stats_advanced_table = team_stats_advanced_table[team_stats_advanced_table['Tm'] != 'Tm']
            # Rename Tm column to Team
            team_stats_advanced_table = team_stats_advanced_table.rename(columns={'Tm': 'Team'})
        else:
            team_stats_advanced_table = team_stats_advanced_table[team_stats_advanced_table['Team'] != 'Team']
        team_stats_advanced_table['Year'] = year
        team_stats_advanced_table['Playoffs_S'] = True
        dfs_team_stats_advanced.append(team_stats_advanced_table)
        
team_stats_advanced = pd.concat(dfs_team_stats_advanced)
team_stats_advanced.head()

,Rk,Team,Age,W,L,PW,PL,MOV,SOS,SRS,...,Unnamed: 27,Arena,Attend.,Attend./G,Year,Playoffs,Playoffs_S,W/L%,Unnamed: 15,Unnamed: 20
0,1.0,Boston Celtics,NaN,22.0,38.0,15.0,45.0,-4.88,0.44,-4.45,...,NaN,Boston Garden,108240.0,NaN,1947,False,False,NaN,NaN,NaN
1,2.0,Chicago Stags,NaN,39.0,22.0,41.0,20.0,3.67,-0.30,3.34,...,NaN,Chicago Stadium,NaN,NaN,1947,True,False,NaN,NaN,NaN
2,3.0,Cleveland Rebels,NaN,30.0,30.0,27.0,33.0,-0.95,0.08,-0.86,...,NaN,Cleveland Arena,NaN,NaN,1947,True,False,NaN,NaN,NaN
3,4.0,Detroit Falcons,NaN,20.0,40.0,24.0,36.0,-2.00,0.18,-1.83,...,NaN,Detroit Olympia,NaN,NaN,1947,False,False,NaN,NaN,NaN
4,5.0,New York Knicks,NaN,33.0,27.0,32.0,28.0,0.65,-0.07,0.58,...,NaN,Madison Square Garden (III),NaN,505.0,1947,True,False,NaN,NaN,NaN


In [132]:
# Remove all columns that have Unnamed in the name
team_stats_advanced = team_stats_advanced.loc[:, ~team_stats_advanced.columns.str.contains('^Unnamed')]
# Remove Rk column
team_stats_advanced = team_stats_advanced.drop(columns=['Rk'])
team_stats_advanced.to_csv('team_stats_advanced.csv', index=False)

## Awards

### Individual

In [133]:
url_player_awards_mvp = "https://www.basketball-reference.com/awards/mvp.html"
url_player_awards_rookie = "https://www.basketball-reference.com/awards/roy.html"
url_player_awards_defensive = "https://www.basketball-reference.com/awards/dpoy.html"
url_player_awards_improved = "https://www.basketball-reference.com/awards/mip.html"
url_player_awards_sixth = "https://www.basketball-reference.com/awards/smoy.html"
url_player_awards_finals_mvp = "https://www.basketball-reference.com/awards/finals_mvp.html"

In [135]:
data = requests.get(url_player_awards_mvp)
with open ('awards/individual/mvp.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_player_awards_rookie)
with open ('awards/individual/rookie.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_player_awards_defensive)
with open ('awards/individual/defensive.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_player_awards_improved)
with open ('awards/individual/improved.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_player_awards_sixth)
with open ('awards/individual/sixth.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_player_awards_finals_mvp)
with open ('awards/individual/finals_mvp.html', 'w') as f:
    f.write(data.text)

In [16]:
with open('awards/individual/mvp.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
soup.find('tr', class_='over_header').decompose()
mvp_html = soup.find(id="mvp_summary")
mvp_table = pd.read_html(str(mvp_html))[0]
mvp_NBA_html = soup.find(id="mvp_NBA")
mvp_NBA_table = pd.read_html(str(mvp_NBA_html))[0]
mvp_ABA_html = soup.find(id="mvp_ABA")
mvp_ABA_table = pd.read_html(str(mvp_ABA_html))[0]
# For every unique player in the MVP table, add a row showing the years he obtained it 
for index, row in mvp_table.iterrows():
    player = row['Player']
    league = row['Lg']
    if (league == 'NBA'):
        years = mvp_NBA_table[mvp_NBA_table['Player'] == player]['Season'].tolist()
    elif (league == 'ABA'):
        years = mvp_ABA_table[mvp_ABA_table['Player'] == player]['Season'].tolist()
    string = ', '.join(str(year) for year in years)
    mvp_table.loc[index, 'MVP_Seasons'] = string
# Rename count to MVP_Count
mvp_table = mvp_table.rename(columns={'Count': 'MVP_Count'})
# Remove asterisk from player names
mvp_table['Player'] = mvp_table['Player'].str.replace('*', '')
dfs_awards_individual = mvp_table

with open('awards/individual/rookie.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
soup.find('tr', class_='over_header').decompose()
rookie_NBA_html = soup.find(id="roy_NBA")
rookie_NBA_table = pd.read_html(str(rookie_NBA_html))[0]
rookie_ABA_html = soup.find(id="roy_ABA")
rookie_ABA_table = pd.read_html(str(rookie_ABA_html))[0]
# Reduce tables to only Season, Lg, Player
rookie_NBA_table = rookie_NBA_table[['Season', 'Lg', 'Player']]
rookie_ABA_table = rookie_ABA_table[['Season', 'Lg', 'Player']]
# Rename Season to ROY_Season
rookie_NBA_table = rookie_NBA_table.rename(columns={'Season': 'ROY_Season'})
rookie_ABA_table = rookie_ABA_table.rename(columns={'Season': 'ROY_Season'})
# Merge NBA and ABA tables
rookie_table = pd.concat([rookie_NBA_table, rookie_ABA_table])
# Remove asterisk from player names
rookie_table['Player'] = rookie_table['Player'].str.replace('*', '')

# Natural join the awards table with the Rookie table on Player and Lg
dfs_awards_individual = pd.merge(dfs_awards_individual, rookie_table, how='outer', on=['Player', 'Lg'])
            
with open('awards/individual/defensive.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
defensive_NBA_html = soup.find(id="dpoy_NBA")
defensive_NBA_table = pd.read_html(str(defensive_NBA_html))[0]
defensive_summary_html = soup.find(id="dpoy_summary")
defensive_summary_table = pd.read_html(str(defensive_summary_html))[0]
# For every unique player in the MVP table, add a row showing the years he obtained it
for index, row in defensive_summary_table.iterrows():
    player = row['Player']
    years = defensive_NBA_table[defensive_NBA_table['Player'] == player]['Season'].tolist()
    string = ', '.join(str(year) for year in years)
    defensive_summary_table.loc[index, 'DPOY_Seasons'] = string
# Rename count to DPOY_Count
defensive_summary_table = defensive_summary_table.rename(columns={'Count': 'DPOY_Count'})
# Remove asterisk from player names
defensive_summary_table['Player'] = defensive_summary_table['Player'].str.replace('*', '')
# Natural join the awards table with the Defensive table on Player
dfs_awards_individual = pd.merge(dfs_awards_individual, defensive_summary_table, how='outer', on=['Player', 'Lg'])

with open('awards/individual/improved.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
improved_NBA_html = soup.find(id="mip_NBA")
improved_NBA_table = pd.read_html(str(improved_NBA_html))[0]
improved_summary_html = soup.find(id="mip_summary")
improved_summary_table = pd.read_html(str(improved_summary_html))[0]
# For every unique player in the MVP table, add a row showing the years he obtained it
for index, row in improved_summary_table.iterrows():
    player = row['Player']
    years = improved_NBA_table[improved_NBA_table['Player'] == player]['Season'].tolist()
    string = ', '.join(str(year) for year in years)
    improved_summary_table.loc[index, 'MIP_Seasons'] = string
# Rename count to MIP_Count
improved_summary_table = improved_summary_table.rename(columns={'Count': 'MIP_Count'})
# Remove asterisk from player names
improved_summary_table['Player'] = improved_summary_table['Player'].str.replace('*', '')
# Natural join the awards table with the Improved table on Player
dfs_awards_individual = pd.merge(dfs_awards_individual, improved_summary_table, how='outer', on=['Player', 'Lg'])

with open('awards/individual/sixth.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
sixth_NBA_html = soup.find(id="smoy_NBA")
sixth_NBA_table = pd.read_html(str(sixth_NBA_html))[0]
sixth_summary_html = soup.find(id="smoy_summary")
sixth_summary_table = pd.read_html(str(sixth_summary_html))[0]
# For every unique player in the MVP table, add a row showing the years he obtained it
for index, row in sixth_summary_table.iterrows():
    player = row['Player']
    years = sixth_NBA_table[sixth_NBA_table['Player'] == player]['Season'].tolist()
    string = ', '.join(str(year) for year in years)
    sixth_summary_table.loc[index, 'SMOY_Seasons'] = string
# Rename count to SMOY_Count
sixth_summary_table = sixth_summary_table.rename(columns={'Count': 'SMOY_Count'})
# Remove asterisk from player names
sixth_summary_table['Player'] = sixth_summary_table['Player'].str.replace('*', '')
# Natural join the awards table with the Sixth table on Player
dfs_awards_individual = pd.merge(dfs_awards_individual, sixth_summary_table, how='outer', on=['Player', 'Lg'])

with open('awards/individual/finals_mvp.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
finals_mvp_NBA_html = soup.find(id="finals_mvp_NBA")
finals_mvp_NBA_table = pd.read_html(str(finals_mvp_NBA_html))[0]
finals_mvp_summary_html = soup.find(id="finals_mvp_summary")
finals_mvp_summary_table = pd.read_html(str(finals_mvp_summary_html))[0]
# For every unique player in the MVP table, add a row showing the years he obtained it
for index, row in finals_mvp_summary_table.iterrows():
    player = row['Player']
    years = finals_mvp_NBA_table[finals_mvp_NBA_table['Player'] == player]['Season'].tolist()
    string = ', '.join(str(year) for year in years)
    finals_mvp_summary_table.loc[index, 'Finals_MVP_Seasons'] = string
# Rename count to Finals_MVP_Count
finals_mvp_summary_table = finals_mvp_summary_table.rename(columns={'Count': 'Finals_MVP_Count'})
# Remove asterisk from player names
finals_mvp_summary_table['Player'] = finals_mvp_summary_table['Player'].str.replace('*', '')
# Natural join the awards table with the Finals MVP table on Player

dfs_awards_individual = pd.merge(dfs_awards_individual, finals_mvp_summary_table, how='outer', on=['Player', 'Lg'])


In [17]:
dfs_awards_individual.to_csv('awards_individual2.csv', index=False)

### Team-Selection

In [136]:
url_team_selection_all_league = "https://www.basketball-reference.com/awards/all_league.html"
url_team_selection_all_rookie = "https://www.basketball-reference.com/awards/all_rookie.html"
url_team_selection_all_defensive = "https://www.basketball-reference.com/awards/all_defense.html"

In [137]:
data = requests.get(url_team_selection_all_league)
with open ('awards/team-selection/all_league.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_team_selection_all_rookie)
with open ('awards/team-selection/all_rookie.html', 'w') as f:
    f.write(data.text)
data = requests.get(url_team_selection_all_defensive)
with open ('awards/team-selection/all_defensive.html', 'w') as f:
    f.write(data.text)

In [18]:
dfs_teams = []

with open('awards/team-selection/all_league.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
all_league_html = soup.find(id='awards_all_league')
all_league_table = pd.read_html(str(all_league_html))[0]
# Combine columns 4 through 8 into one column
all_league_table['Players'] = all_league_table.iloc[:, 4:9].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
# Drop columns 4 through 8
all_league_table.drop(all_league_table.columns[4:9], axis=1, inplace=True)
# Drop all rows that have NaN in Season column
all_league_table = all_league_table.dropna(subset=['Season'])
all_league_table['Type'] = 'All League'
dfs_teams.append(all_league_table)

with open('awards/team-selection/all_rookie.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
all_rookie_html = soup.find(id='awards_all_rookie')
all_rookie_table = pd.read_html(str(all_rookie_html))[0]
# Combine columns 4 through 8 into one column
all_rookie_table['Players'] = all_rookie_table.iloc[:, 3:8].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
# Drop columns 4 through 8
all_rookie_table.drop(all_rookie_table.columns[3:8], axis=1, inplace=True)
# Drop all rows that have NaN in Season column
all_rookie_table = all_rookie_table.dropna(subset=['Season'])
all_rookie_table['Type'] = 'All Rookie'
dfs_teams.append(all_rookie_table)

with open('awards/team-selection/all_defensive.html') as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
all_defense_html = soup.find(id='awards_all_defense')
all_defense_table = pd.read_html(str(all_defense_html))[0]
# Combine columns 4 through 8 into one column
all_defense_table['Players'] = all_defense_table.iloc[:, 4:9].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)
# Drop columns 4 through 8
all_defense_table.drop(all_defense_table.columns[4:9], axis=1, inplace=True)
# Drop all rows that have NaN in Season column
all_defense_table = all_defense_table.dropna(subset=['Season'])
all_defense_table['Type'] = 'All Defensive'
dfs_teams.append(all_defense_table)

team_selections = pd.concat(dfs_teams)
# Delete the Voting column
team_selections.drop(columns=['Voting'], inplace=True)
# Remove rows that have Players as Nan or empty string
team_selections = team_selections[team_selections['Players'].notna()]
team_selections = team_selections[team_selections['Players'] != '']

team_selections.head()

,Season,Lg,Tm,Players,Type
0,2022-23,NBA,1st,"Joel Embiid C, Giannis Antetokounmpo F, Jayson...",All League
1,2022-23,NBA,2nd,"Nikola Jokić C, Jimmy Butler F, Jaylen Brown F...",All League
2,2022-23,NBA,3rd,"Domantas Sabonis C, LeBron James F, Julius Ran...",All League
4,2021-22,NBA,1st,"Nikola Jokić C, Giannis Antetokounmpo F, Jayso...",All League
5,2021-22,NBA,2nd,"Joel Embiid C, Kevin Durant F, DeMar DeRozan F...",All League


In [19]:
team_selections.to_csv('team_selections2.csv', index=False)

### Championships

In [138]:
url_team_championships = "https://www.basketball-reference.com/playoffs/"

In [139]:
data = requests.get(url_team_championships)
with open ('awards/championships.html', 'w') as f:
    f.write(data.text)

In [143]:
with open("awards/championships.html") as f:
    page = f.read()
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_='over_header').decompose()
championships_html = soup.find(id="champions_index")
championships_table = pd.read_html(str(championships_html))[0]
# Only keep Year, Lg, and Champion columns
championships_table = championships_table[['Year', 'Lg', 'Champion']]
championships_table.head()

,Year,Lg,Champion
0,2023.0,NBA,Denver Nuggets
1,2022.0,NBA,Golden State Warriors
2,2021.0,NBA,Milwaukee Bucks
3,2020.0,NBA,Los Angeles Lakers
4,2019.0,NBA,Toronto Raptors


In [155]:
championships_table.to_csv('championships.csv', index=False)

### Forecast

In [ ]:
data = pd.read_csv('csvs/team_forecast.csv')
# Make row 1 the header
data.columns = data.iloc[1]
# Remove rows 0 through 2
data = data.iloc[2:]
# Remove first column
data = data.iloc[:, 1:]
# Keep only first, and last 3 columns
data = data.iloc[:, [0, -3, -2, -1]]
# Update csv
data.to_csv('csvs/team_forecast.csv', index=False)
print(data)

In [221]:
data = pd.read_csv('csvs/team_forecast.csv')
# Find values in last column that are "<.1" and replace with 0
data['Champ%'] = data['Champ%'].str.replace('<0.1', '0')
# Update csv
data.to_csv('csvs/team_forecast.csv', index=False)